# Unsupervised Arabic Sentiment Analysis

This notebook analyzes Arabic text sentiment **without relying on potentially incorrect labels**.

## Approaches:
1. **Lexicon-based**: Using Arabic sentiment word dictionaries
2. **Rule-based**: Pattern matching for positive/negative expressions
3. **Clustering**: Grouping similar texts to discover sentiment patterns
4. **Weak supervision**: Creating rules from text patterns

## 1. Install and Import Libraries

In [1]:
# Install required packages
%pip install -q kagglehub nltk scikit-learn pandas numpy camel-tools ipywidgets

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [2]:
import kagglehub
import nltk
import pandas as pd
import numpy as np
import re
import os
from collections import Counter, defaultdict

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## 2. Download Dataset

In [3]:
# Download Arabic classification dataset
path = kagglehub.dataset_download("saurabhshahane/arabic-classification")
print(f"Dataset downloaded to: {path}")

# Load the CSV file
csv_file_path = os.path.join(path, 'arabic_dataset_classifiction.csv')
if os.path.isdir(csv_file_path):
    files = [f for f in os.listdir(csv_file_path) if f.endswith('.csv')]
    if files:
        csv_file_path = os.path.join(csv_file_path, files[0])

df = pd.read_csv(csv_file_path)
print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
df.head()

Dataset downloaded to: C:\Users\pc\.cache\kagglehub\datasets\saurabhshahane\arabic-classification\versions\1

Dataset shape: (111728, 2)
Columns: ['text', 'targe']


,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0


## 3. Text Preprocessing

In [4]:
# Download NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
from nltk.corpus import stopwords

arabic_stopwords = set(stopwords.words('arabic'))

def preprocess_arabic_text(text, remove_stopwords=True):
    """Clean and normalize Arabic text"""
    if pd.isna(text) or text == '':
        return ''
    
    # Remove diacritics
    text = re.sub(r'[\u064B-\u0652\u0653\u0670]', '', text)
    
    # Remove Tatweel
    text = re.sub(r'\u0640', '', text)
    
    # Normalize characters
    text = re.sub(r'[\u0622\u0623\u0625]', '\u0627', text)  # Alif forms
    text = re.sub(r'\u0649', '\u064a', text)  # Alef Maksura to Yeh
    text = re.sub(r'\u0629', '\u0647', text)  # Ta Marbuta to Ha
    
    # Keep only Arabic letters and spaces
    text = re.sub(r'[^\u0621-\u064A\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize and optionally remove stopwords
    tokens = text.split()
    if remove_stopwords:
        tokens = [w for w in tokens if w not in arabic_stopwords and len(w) > 1]
    
    return ' '.join(tokens)

# Apply preprocessing
print("Preprocessing text...")
df['cleaned_text'] = df['text'].apply(preprocess_arabic_text)
df = df[df['cleaned_text'].str.len() > 0].reset_index(drop=True)

print(f"✓ Processed {len(df)} texts")
df[['text', 'cleaned_text']].head()

Preprocessing text...
✓ Processed 108787 texts


,text,cleaned_text
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,استوديوهات ورزازات وصحراء مرزوكه واثار وليلي ا...
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,قررت النجمه الامريكيه اوبرا وينفري الا يقتصر ع...
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,اخبارنا المغربيه الوزاني تصوير الشملالي الهب ا...
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,اخبارنا المغربيه قال ابراهيم الراشدي محامي سعد...
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,تزال صناعه الجلود المغرب تتبع الطريقه التقليدي...


## 4. Build Arabic Sentiment Lexicon

We'll create sentiment lexicons by analyzing word co-occurrence patterns in the text.

In [5]:
# Common Arabic positive and negative seed words
# These are starting points that we'll expand

POSITIVE_SEEDS = {
    'رائع', 'ممتاز', 'جميل', 'عظيم', 'مذهل', 'سعيد', 'حب', 'احب',
    'افضل', 'جيد', 'مفيد', 'ناجح', 'نجاح', 'سرور', 'بهجه', 'فرح',
    'ممل', 'مشوق', 'رائه', 'حلو', 'لطيف', 'جذاب', 'مثالي',
    'احسن', 'اجمل', 'اروع', 'يستحق', 'مميز', 'فائق', 'متميز'
}

NEGATIVE_SEEDS = {
    'سيء', 'سيي', 'فشل', 'فاشل', 'سيئه', 'غضب', 'حزن', 'حزين',
    'كره', 'اكره', 'مشكله', 'خطا', 'خطأ', 'ضعيف', 'مروع', 'فظيع',
    'رديء', 'ردي', 'اسوء', 'بشع', 'مقرف', 'مزعج', 'محبط',
    'سلبي', 'كارثه', 'كريه', 'مريع', 'مخيب', 'غير', 'لم', 'ليس'
}

INTENSIFIERS = {
    'جدا', 'للغايه', 'كثيرا', 'حقا', 'فعلا', 'تماما', 'ابدا'
}

NEGATIONS = {
    'لا', 'ليس', 'لم', 'لن', 'غير', 'بدون', 'ما'
}

print(f"Positive seed words: {len(POSITIVE_SEEDS)}")
print(f"Negative seed words: {len(NEGATIVE_SEEDS)}")
print(f"Intensifiers: {len(INTENSIFIERS)}")
print(f"Negations: {len(NEGATIONS)}")

Positive seed words: 30
Negative seed words: 31
Intensifiers: 7
Negations: 7


In [6]:
def analyze_text_sentiment_lexicon(text, positive_words, negative_words, 
                                   intensifiers, negations):
    """
    Analyze sentiment using lexicon-based approach.
    Returns sentiment score and explanation.
    """
    tokens = text.split()
    
    positive_score = 0
    negative_score = 0
    
    positive_matches = []
    negative_matches = []
    
    i = 0
    while i < len(tokens):
        token = tokens[i]
        
        # Check for negation + sentiment word patterns
        if i < len(tokens) - 1:
            next_token = tokens[i + 1]
            
            # Negation flips sentiment
            if token in negations:
                if next_token in positive_words:
                    negative_score += 1
                    negative_matches.append(f"{token} {next_token}")
                    i += 2
                    continue
                elif next_token in negative_words:
                    positive_score += 0.5  # Double negative
                    positive_matches.append(f"{token} {next_token}")
                    i += 2
                    continue
            
            # Intensifier amplifies sentiment
            if next_token in intensifiers:
                if token in positive_words:
                    positive_score += 2
                    positive_matches.append(f"{token} {next_token}")
                    i += 2
                    continue
                elif token in negative_words:
                    negative_score += 2
                    negative_matches.append(f"{token} {next_token}")
                    i += 2
                    continue
        
        # Single word sentiment
        if token in positive_words:
            positive_score += 1
            positive_matches.append(token)
        elif token in negative_words:
            negative_score += 1
            negative_matches.append(token)
        
        i += 1
    
    # Calculate final score
    net_score = positive_score - negative_score
    
    # Determine sentiment category
    if net_score >= 2:
        sentiment = 'Positive'
        confidence = min(abs(net_score) / 5, 1.0)
    elif net_score >= 1:
        sentiment = 'Somewhat Positive'
        confidence = 0.6
    elif net_score <= -2:
        sentiment = 'Negative'
        confidence = min(abs(net_score) / 5, 1.0)
    elif net_score <= -1:
        sentiment = 'Somewhat Negative'
        confidence = 0.6
    else:
        sentiment = 'Neutral'
        confidence = 0.5
    
    return {
        'sentiment': sentiment,
        'score': net_score,
        'positive_score': positive_score,
        'negative_score': negative_score,
        'confidence': confidence,
        'positive_matches': positive_matches,
        'negative_matches': negative_matches
    }

# Test on sample texts
samples = [
    "هذا المنتج رائع جدا",
    "الخدمه سيئه للغايه",
    "ليس سيئا",
    "جيد جدا واحبه",
    "لا احب هذا المنتج"
]

print("Testing lexicon-based sentiment analysis:\n")
for sample in samples:
    cleaned = preprocess_arabic_text(sample, remove_stopwords=False)
    result = analyze_text_sentiment_lexicon(
        cleaned, POSITIVE_SEEDS, NEGATIVE_SEEDS, 
        INTENSIFIERS, NEGATIONS
    )
    print(f"Text: {sample}")
    print(f"  → {result['sentiment']} (score: {result['score']:+d}, confidence: {result['confidence']:.2f})")
    if result['positive_matches']:
        print(f"    ✅ Positive: {', '.join(result['positive_matches'])}")
    if result['negative_matches']:
        print(f"    ❌ Negative: {', '.join(result['negative_matches'])}")
    print()

Testing lexicon-based sentiment analysis:

Text: هذا المنتج رائع جدا
  → Positive (score: +2, confidence: 0.40)
    ✅ Positive: رائع جدا

Text: الخدمه سيئه للغايه
  → Negative (score: -2, confidence: 0.40)
    ❌ Negative: سيئه للغايه

Text: ليس سيئا
  → Somewhat Negative (score: -1, confidence: 0.60)
    ❌ Negative: ليس

Text: جيد جدا واحبه
  → Positive (score: +2, confidence: 0.40)
    ✅ Positive: جيد جدا

Text: لا احب هذا المنتج
  → Somewhat Negative (score: -1, confidence: 0.60)
    ❌ Negative: لا احب



## 5. Apply Lexicon-Based Analysis to Dataset

In [7]:
# Analyze all texts
print("Analyzing sentiment for all texts...")

results = []
for text in df['cleaned_text']:
    result = analyze_text_sentiment_lexicon(
        text, POSITIVE_SEEDS, NEGATIVE_SEEDS,
        INTENSIFIERS, NEGATIONS
    )
    results.append(result)

# Add results to dataframe
df['lexicon_sentiment'] = [r['sentiment'] for r in results]
df['lexicon_score'] = [r['score'] for r in results]
df['lexicon_confidence'] = [r['confidence'] for r in results]
df['positive_matches'] = [', '.join(r['positive_matches']) for r in results]
df['negative_matches'] = [', '.join(r['negative_matches']) for r in results]

print("✓ Analysis complete\n")

# Show distribution
print("Sentiment Distribution (Lexicon-based):")
print(df['lexicon_sentiment'].value_counts())

print("\nAverage confidence by sentiment:")
print(df.groupby('lexicon_sentiment')['lexicon_confidence'].mean().round(2))

# Show some examples
print("\nSample results:")
df[['text', 'lexicon_sentiment', 'lexicon_score', 'positive_matches', 'negative_matches']].head(10)

Analyzing sentiment for all texts...
✓ Analysis complete

Sentiment Distribution (Lexicon-based):
lexicon_sentiment
Neutral              81575
Somewhat Positive    10651
Somewhat Negative     8320
Positive              5081
Negative              3160
Name: count, dtype: int64

Average confidence by sentiment:
lexicon_sentiment
Negative             0.52
Neutral              0.50
Positive             0.54
Somewhat Negative    0.60
Somewhat Positive    0.60
Name: lexicon_confidence, dtype: float64

Sample results:


,text,lexicon_sentiment,lexicon_score,positive_matches,negative_matches
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,Neutral,0.0,,
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,Neutral,0.0,,
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,Somewhat Positive,1.0,سعيد,
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,Neutral,0.0,,
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,Neutral,0.0,,
5,مصطفى الصوفي انطلقت أمس فعاليات الدورة الثالثة...,Somewhat Positive,1.0,احسن,
6,الاتحاد الاشتراكي تحيي الفنانة اللبنانية نجوى ...,Neutral,0.0,,
7,أقدمت كاثريونا وايت حبيبة الممثل جيم كاري على ...,Neutral,0.0,,
8,أحمد الريدي تطور جديد شهدته قضية الفنانة زينة ...,Neutral,0.0,,
9,أخبارنا المغربية بشراكة بين مؤسسة البشير للتعل...,Neutral,0.0,,


## 6. Discover New Sentiment Words

Use co-occurrence analysis to find words that frequently appear with positive/negative seeds.

In [8]:
def discover_sentiment_words(texts, seed_positive, seed_negative, top_n=30):
    """
    Discover new sentiment words based on co-occurrence with seed words.
    """
    positive_cooccur = Counter()
    negative_cooccur = Counter()
    
    for text in texts:
        tokens = text.split()
        
        # Check if text contains seed words
        has_positive = any(token in seed_positive for token in tokens)
        has_negative = any(token in seed_negative for token in tokens)
        
        # Count co-occurring words
        if has_positive:
            for token in tokens:
                if token not in seed_positive and token not in seed_negative:
                    positive_cooccur[token] += 1
        
        if has_negative:
            for token in tokens:
                if token not in seed_positive and token not in seed_negative:
                    negative_cooccur[token] += 1
    
    # Find words that strongly associate with one sentiment
    discovered_positive = []
    discovered_negative = []
    
    all_words = set(positive_cooccur.keys()) | set(negative_cooccur.keys())
    
    for word in all_words:
        pos_count = positive_cooccur.get(word, 0)
        neg_count = negative_cooccur.get(word, 0)
        total = pos_count + neg_count
        
        if total < 5:  # Filter rare words
            continue
        
        # Calculate sentiment ratio
        if pos_count > neg_count * 2:  # Strongly positive
            discovered_positive.append((word, pos_count, pos_count / total))
        elif neg_count > pos_count * 2:  # Strongly negative
            discovered_negative.append((word, neg_count, neg_count / total))
    
    # Sort by count
    discovered_positive.sort(key=lambda x: x[1], reverse=True)
    discovered_negative.sort(key=lambda x: x[1], reverse=True)
    
    return discovered_positive[:top_n], discovered_negative[:top_n]

# Discover new sentiment words
print("Discovering new sentiment words from the corpus...\n")
new_positive, new_negative = discover_sentiment_words(
    df['cleaned_text'], 
    POSITIVE_SEEDS, 
    NEGATIVE_SEEDS,
    top_n=30
)

print("Discovered Positive Words:")
print("Word\t\tCount\tRatio")
print("-" * 40)
for word, count, ratio in new_positive[:15]:
    print(f"{word:15s}\t{count}\t{ratio:.2f}")

print("\nDiscovered Negative Words:")
print("Word\t\tCount\tRatio")
print("-" * 40)
for word, count, ratio in new_negative[:15]:
    print(f"{word:15s}\t{count}\t{ratio:.2f}")

Discovering new sentiment words from the corpus...

Discovered Positive Words:
Word		Count	Ratio
----------------------------------------
العربي         	3368	0.67
العربيه        	3285	0.67
الفنان         	2902	0.83
مجال           	2848	0.71
البرنامج       	2541	0.69
الفنيه         	2490	0.79
ملال           	2418	0.73
الفيلم         	2405	0.80
عمل            	2309	0.67
فيلم           	2254	0.83
الفني          	2245	0.73
بني            	2128	0.71
المهرجان       	1991	0.86
اغنيه          	1963	0.86
جائزه          	1875	0.85

Discovered Negative Words:
Word		Count	Ratio
----------------------------------------
القدم          	18528	0.67
جامعه          	3358	0.76
لقجع           	2039	0.70
والرياضه       	1556	0.67
السله          	1236	0.75
المديري        	1011	0.74
المتقي         	934	0.67
الفهري         	692	0.75
الفيفا         	643	0.69
بالجامعه       	570	0.68
فرع            	566	0.67
النسويه        	559	0.70
اوزين          	550	0.68
الشغب          	526	0.70
بلاتر          	472	0.73


## 7. Expand Lexicon with Discovered Words

In [9]:
# Expand lexicon with high-confidence discovered words
EXPANDED_POSITIVE = POSITIVE_SEEDS.copy()
EXPANDED_NEGATIVE = NEGATIVE_SEEDS.copy()

# Add discovered words with ratio > 0.7
for word, count, ratio in new_positive:
    if ratio > 0.7 and count > 10:
        EXPANDED_POSITIVE.add(word)

for word, count, ratio in new_negative:
    if ratio > 0.7 and count > 10:
        EXPANDED_NEGATIVE.add(word)

print(f"Expanded positive lexicon: {len(POSITIVE_SEEDS)} → {len(EXPANDED_POSITIVE)}")
print(f"Expanded negative lexicon: {len(NEGATIVE_SEEDS)} → {len(EXPANDED_NEGATIVE)}")

print("\nNew positive words added:")
print(EXPANDED_POSITIVE - POSITIVE_SEEDS)

print("\nNew negative words added:")
print(EXPANDED_NEGATIVE - NEGATIVE_SEEDS)

Expanded positive lexicon: 30 → 52
Expanded negative lexicon: 31 → 47

New positive words added:
{'جائزه', 'الفني', 'الفنان', 'ملال', 'الفنيه', 'المهرجان', 'حسبان', 'الفيلم', 'الاغنيه', 'الفنانين', 'بفضل', 'فيلم', 'بني', 'الاعمال', 'الفنانه', 'السيد', 'الموسيقي', 'مجال', 'مهرجان', 'السينما', 'اغنيه', 'المخرج'}

New negative words added:
{'لجامعه', 'السله', 'الاتحادات', 'عرضيه', 'الفهري', 'العصب', 'جامعه', 'بلاتر', 'دومو', 'غايبي', 'المديري', 'حياتو', 'السنوسي', 'جامعي', 'لقجع', 'النسويه'}


## 8. Re-analyze with Expanded Lexicon

In [10]:
# Re-analyze with expanded lexicon
print("Re-analyzing with expanded lexicon...")

results_v2 = []
for text in df['cleaned_text']:
    result = analyze_text_sentiment_lexicon(
        text, EXPANDED_POSITIVE, EXPANDED_NEGATIVE,
        INTENSIFIERS, NEGATIONS
    )
    results_v2.append(result)

# Add new results
df['final_sentiment'] = [r['sentiment'] for r in results_v2]
df['final_score'] = [r['score'] for r in results_v2]
df['final_confidence'] = [r['confidence'] for r in results_v2]

print("✓ Re-analysis complete\n")

print("Sentiment Distribution (Expanded Lexicon):")
print(df['final_sentiment'].value_counts())

print("\nComparison: Original vs Expanded")
comparison = pd.crosstab(df['lexicon_sentiment'], df['final_sentiment'])
print(comparison)

Re-analyzing with expanded lexicon...
✓ Re-analysis complete

Sentiment Distribution (Expanded Lexicon):
final_sentiment
Neutral              60350
Positive             20334
Somewhat Positive    14225
Somewhat Negative     7383
Negative              6495
Name: count, dtype: int64

Comparison: Original vs Expanded
final_sentiment    Negative  Neutral  Positive  Somewhat Negative  \
lexicon_sentiment                                                   
Negative               2807       74        64                183   
Neutral                1586    59201     10470               2149   
Positive                 20       25      4933                 22   
Somewhat Negative      1971      763       415               4902   
Somewhat Positive       111      287      4452                127   

final_sentiment    Somewhat Positive  
lexicon_sentiment                     
Negative                          32  
Neutral                         8169  
Positive                          81  
Somew

## 9. Clustering-Based Sentiment Discovery

In [11]:
# Use TF-IDF + K-Means to discover sentiment clusters
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

print("Performing clustering analysis...")

# Create TF-IDF features
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['cleaned_text'])

# Reduce dimensionality for visualization
svd = TruncatedSVD(n_components=50, random_state=42)
X_reduced = svd.fit_transform(X)

# Cluster into 5 groups (matching sentiment categories)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_reduced)

df['cluster'] = clusters

print(f"✓ Clustering complete\n")
print("Cluster distribution:")
print(df['cluster'].value_counts().sort_index())

# Analyze each cluster
print("\nCluster Analysis:")
for cluster_id in range(5):
    cluster_texts = df[df['cluster'] == cluster_id]['cleaned_text']
    cluster_sentiments = df[df['cluster'] == cluster_id]['final_sentiment']
    
    # Get most common words in cluster
    all_words = ' '.join(cluster_texts).split()
    word_freq = Counter(all_words).most_common(10)
    
    print(f"\nCluster {cluster_id} (n={len(cluster_texts)}):")
    print(f"  Most common sentiment: {cluster_sentiments.mode()[0] if len(cluster_sentiments) > 0 else 'N/A'}")
    print(f"  Top words: {', '.join([w for w, c in word_freq])}")

Performing clustering analysis...
✓ Clustering complete

Cluster distribution:
cluster
0    13639
1    17405
2    10183
3    46721
4    20839
Name: count, dtype: int64

Cluster Analysis:

Cluster 0 (n=13639):
  Most common sentiment: Neutral
  Top words: علي, الي, ان, القضائيه, العامه, عناصر, المتهم, انه, الشرطه, النيابه

Cluster 1 (n=17405):
  Most common sentiment: Neutral
  Top words: ان, علي, الي, المنتخب, الوطني, القدم, كاس, لكره, المباراه, مباراه

Cluster 2 (n=10183):
  Most common sentiment: Neutral
  Top words: ان, علي, الي, الحكومه, حزب, الحزب, بنكيران, رئيس, والتنميه, العداله

Cluster 3 (n=46721):
  Most common sentiment: Neutral
  Top words: ان, علي, الي, المغرب, خلال, المغربيه, انه, او, المائه, المغربي

Cluster 4 (n=20839):
  Most common sentiment: Neutral
  Top words: ان, علي, الي, الفريق, الرياضي, المباراه, الرجاء, الموسم, مباراه, امام


## 10. Comparison with Original Labels

In [12]:
# Map original labels to sentiment names
sentiment_mapping = {
    0: 'Negative',
    1: 'Somewhat Negative',
    2: 'Neutral',
    3: 'Somewhat Positive',
    4: 'Positive'
}

df['original_label'] = df['targe'].map(sentiment_mapping)

# Compare
print("Comparison: Our Lexicon-Based Analysis vs Original Labels\n")

# Agreement percentage
agreement = (df['final_sentiment'] == df['original_label']).mean()
print(f"Agreement rate: {agreement:.2%}")

# Confusion matrix
print("\nCross-tabulation:")
crosstab = pd.crosstab(
    df['final_sentiment'], 
    df['original_label'],
    margins=True
)
print(crosstab)

# Show examples of disagreement
print("\nExamples where our analysis differs from original labels:")
disagreements = df[df['final_sentiment'] != df['original_label']].head(10)
for idx, row in disagreements.iterrows():
    print(f"\nText: {row['text']}")
    print(f"  Original label: {row['original_label']}")
    print(f"  Our analysis: {row['final_sentiment']} (score: {row['final_score']:+.0f})")
    if row['positive_matches']:
        print(f"  Positive words: {row['positive_matches']}")
    if row['negative_matches']:
        print(f"  Negative words: {row['negative_matches']}")

Comparison: Our Lexicon-Based Analysis vs Original Labels

Agreement rate: 15.80%

Cross-tabulation:
original_label     Negative  Neutral  Positive  Somewhat Negative  \
final_sentiment                                                     
Negative                 77      100      5873                102   
Neutral                2539     8504     22597              13312   
Positive               9050     2766      4982               1010   
Somewhat Negative       226      397      4935                619   
Somewhat Positive      1844     2398      5288               1685   
All                   13736    14165     43675              16728   

original_label     Somewhat Positive     All  
final_sentiment                               
Negative                         343    6495  
Neutral                        13398   60350  
Positive                        2526   20334  
Somewhat Negative               1206    7383  
Somewhat Positive               3010   14225  
All              

In [13]:
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 11. Interactive Sentiment Analyzer

In [14]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def analyze_custom_text(text):
    """Analyze sentiment of custom Arabic text"""
    cleaned = preprocess_arabic_text(text, remove_stopwords=False)
    
    result = analyze_text_sentiment_lexicon(
        cleaned, EXPANDED_POSITIVE, EXPANDED_NEGATIVE,
        INTENSIFIERS, NEGATIONS
    )
    
    print("="*70)
    print(f"📝 Original Text: {text}")
    print(f"🧹 Cleaned Text: {cleaned}")
    print("-"*70)
    print(f"🎯 Sentiment: {result['sentiment']}")
    print(f"📊 Score: {result['score']:+d} (Positive: +{result['positive_score']}, Negative: -{result['negative_score']})")
    print(f"🔍 Confidence: {result['confidence']:.0%}")
    
    if result['positive_matches']:
        print(f"\n✅ Positive indicators: {', '.join(result['positive_matches'])}")
    
    if result['negative_matches']:
        print(f"❌ Negative indicators: {', '.join(result['negative_matches'])}")
    
    print("="*70)

# Create widget
text_input = widgets.Textarea(
    value='',
    placeholder='أدخل النص العربي هنا...',
    description='النص:',
    layout=widgets.Layout(width='90%', height='100px')
)

analyze_btn = widgets.Button(
    description='🔍 تحليل المشاعر (Analyze)',
    button_style='success'
)

output = widgets.Output()

def on_click(b):
    with output:
        clear_output()
        if text_input.value.strip():
            analyze_custom_text(text_input.value)
        else:
            print("⚠️  الرجاء إدخال نص (Please enter text)")

analyze_btn.on_click(on_click)

print("\n" + "="*70)
print("Interactive Arabic Sentiment Analyzer (Lexicon-Based)")
print("="*70)
display(widgets.VBox([text_input, analyze_btn, output]))


Interactive Arabic Sentiment Analyzer (Lexicon-Based)


## 12. Save Results and Lexicons

In [15]:
import json

# Save expanded lexicons
lexicons = {
    'positive_words': list(EXPANDED_POSITIVE),
    'negative_words': list(EXPANDED_NEGATIVE),
    'intensifiers': list(INTENSIFIERS),
    'negations': list(NEGATIONS)
}

with open('arabic_sentiment_lexicon.json', 'w', encoding='utf-8') as f:
    json.dump(lexicons, f, ensure_ascii=False, indent=2)

# Save analyzed dataset
df.to_csv('arabic_sentiment_analyzed.csv', index=False, encoding='utf-8')

print("✓ Saved files:")
print("  - arabic_sentiment_lexicon.json (sentiment word dictionaries)")
print("  - arabic_sentiment_analyzed.csv (analyzed dataset)")

print(f"\nLexicon Statistics:")
print(f"  Positive words: {len(EXPANDED_POSITIVE)}")
print(f"  Negative words: {len(EXPANDED_NEGATIVE)}")
print(f"  Intensifiers: {len(INTENSIFIERS)}")
print(f"  Negations: {len(NEGATIONS)}")

✓ Saved files:
  - arabic_sentiment_lexicon.json (sentiment word dictionaries)
  - arabic_sentiment_analyzed.csv (analyzed dataset)

Lexicon Statistics:
  Positive words: 52
  Negative words: 47
  Intensifiers: 7
  Negations: 7


## Summary

This notebook demonstrates **unsupervised sentiment analysis** that doesn't rely on potentially incorrect labels:

### ✅ What We Did:

1. **Lexicon-Based Analysis**: Used Arabic sentiment word dictionaries
2. **Pattern Recognition**: Detected negations, intensifiers, and compound expressions
3. **Lexicon Expansion**: Discovered new sentiment words from text patterns
4. **Clustering**: Grouped similar texts to find natural sentiment patterns
5. **Validation**: Compared results with original (unreliable) labels

### 💡 Advantages:

- **No training needed**: Works immediately without labeled data
- **Interpretable**: Shows exactly which words drove the sentiment decision
- **Customizable**: Easy to add domain-specific words
- **Language-aware**: Handles Arabic-specific patterns (negation, intensifiers)

### 🔄 Next Steps:

- Review discovered sentiment words and refine the lexicon
- Add domain-specific terms for your use case
- Use cluster analysis to find mislabeled examples
- Combine lexicon scores with ML models for hybrid approach

In [16]:
# Install necessary libraries
%pip install -qU langchain-google-genai langchain


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


# Retrieve the API key from Colab secrets


# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY)

print("LangChain and Gemini model environment prepared.")

LangChain and Gemini model environment prepared.
